In [1]:
import os
os.chdir('../')

In [2]:
!pip install -U torch
!pip install medialpy

/bin/bash: /home/ale/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


# Import modules and load data

In [3]:
#autoload modules in notebook
%load_ext autoreload
%autoreload 2  

In [4]:
from itertools import combinations
import json
import os
import shutil
import pandas as pd
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import wandb
from transformers import DistilBertTokenizer, TFDistilBertModel, AutoTokenizer, TFAutoModel
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import CLIP_model
import CLIP_data_load

2023-08-17 20:31:53.521650: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

In [ ]:
# Hyperparameters
train_percentage = 0.8
test_percentage = 0.2
validation_percentage = 0.15
latent_dim_imgs = 1024
latent_dim_text = 768
latent_dim_common = 512
batch_size = 100
SEED = 116
img_shape = (128,128,3)
text_input_shape = (128)
bert_model_name = 'distilbert-base-uncased' #bert-base-multilingual-uncased
bert_model_name = "medicalai/ClinicalBERT"
max_len = 128
model_name = 'clip_no_stopwords_our_loss'

# Paths
all_images_path = 'dataset/resized_train/'
project_location = '.'
all_captions_path = f'{project_location}/dataset/caption_prediction_train.csv'  
all_concept_ids_path = f'{project_location}/dataset/concept_detection_train.csv'
all_concepts_path = f'{project_location}/dataset/concepts.csv'

zip_dataset_location = f'{project_location}/dataset/resized_train.zip'
model_loc = f'{project_location}/weights/{model_name}.h5'

In [ ]:
run = wandb.init(project="clip",
           entity='calonca',
           name=model_name,
           config = {
              'batch_size':batch_size,
              'latent_dim_common':latent_dim_common,
              'latent_dim_text':latent_dim_text,
              'latent_dim_imgs':latent_dim_imgs,
              'img_shape':img_shape,
              'text_input_shape':text_input_shape,
              'model_name': model_name,
              'bert_model_name': bert_model_name,
           })

wandb: Currently logged in as: pablo-giaccaglia (calonca). Use `wandb login --relogin` to force relogin


## **PRE TRAINED BERT MODEL**

## **PRE TRAINED FEATURE EXTRACTION NETWORK**

## **CLIP MODEL**

# Data preparation

In [ ]:
#Merging all dataframes toghether into ID, caption, cuis, concepts
captionsDF = pd.read_csv(all_captions_path, sep='\t')
concept_id_df = pd.read_csv(all_concept_ids_path, sep='\t')
concept_df = pd.read_csv(all_concepts_path, sep=',')

concepts = concept_df.set_index('concept').T.to_dict('list')
concept_id_df.cuis = concept_id_df.cuis.apply(lambda x: x.split(';'))
concept_id_df['concepts'] = concept_id_df.cuis.apply(lambda cuis: {concepts[cui][0] for cui in cuis})

captionsDF = pd.merge(captionsDF,concept_id_df, on="ID")
captionsDF.ID = captionsDF.ID.apply(lambda x: x+'.jpg')

# **Data generation**

In [ ]:
train_val, test = train_test_split(captionsDF, test_size = test_percentage, shuffle = True, random_state = SEED)
train, val = train_test_split(train_val, test_size = validation_percentage, shuffle = True, random_state = SEED)

In [ ]:
words_to_exclude = [
    'show',
    'arrow',
    'image',
    'scan',
    'demonstrate',
    'patient',
    'compute',
    'reveal',
    'imaging',
    'see',
    'level',
    'case'
    'note',
    'month',
    'within',
    'indicate',
    'also',
    'without',
    'year',
    'day',
    'followup',
    'follow',
    'finding',
    'well',
    'along',
    'figure',
    'week',
    'obtain',
    'take',
    'main',
    'type',
    'group',
    'defect',
    'admission',
    'leave'
]

In [ ]:
train = CLIP_data_load.caption_clean(train, words_to_remove=words_to_exclude)
val = CLIP_data_load.caption_clean(val,  words_to_remove=words_to_exclude)

[nltk_data] Downloading package stopwords to /home/ale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ale/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ale/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#tokenizer = DistilBertTokenizer.from_pretrained(bert_model_name)
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")

trainList = CLIP_data_load.paths_captions_emb_list(train,  all_images_path,tokenizer=tokenizer,max_len=max_len, remove_images_threshold = 0)
valList = CLIP_data_load.paths_captions_emb_list(val, all_images_path,tokenizer=tokenizer,max_len=max_len, remove_images_threshold = 0)

100%|██████████| 9993/9993 [00:04<00:00, 2206.57it/s]


In [ ]:
train_gen = CLIP_data_load.ClipBaseGenerator(channels_first = False, preprocessing_function = None, data = trainList, batch_size = batch_size, out_shape = img_shape[0:2], shuffle = True,seed=SEED)
val_gen = CLIP_data_load.ClipBaseGenerator(channels_first = False, preprocessing_function = None, data = valList, batch_size = batch_size, out_shape = img_shape[0:2], shuffle = True,seed=SEED)

## **Train CLIP**

In [ ]:
fen_model = tf.keras.applications.convnext.ConvNeXtTiny(
    include_top = False,
    weights="imagenet",
    pooling='avg',
    input_shape = img_shape
)


#fen_model = TFAutoModel.from_pretrained("ngctnnnn/lvmmed_rn50",  from_pt=True)
#img_shape=(3,224,224)
#bert_model = TFDistilBertModel.from_pretrained(bert_model_name)
bert_model = TFAutoModel.from_pretrained("medicalai/ClinicalBERT", from_pt=True)
model = CLIP_model.get_clip_model(
    image_input_shape = img_shape,
    text_input_shape = text_input_shape,
    text_encoder = bert_model,
    image_encoder = fen_model,
    latent_dim_imgs = latent_dim_imgs,
    latent_dim_text = latent_dim_text,
    latent_dim_common = latent_dim_common,
    train_bert = True,
    loss = CLIP_model.custom_loss_with_temp(1),
    learning_rate = 1e-4
)

2023-08-17 19:27:15.920942: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-17 19:27:15.951879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-17 19:27:15.952137: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

111650432/111650432 [==============================] - 26s 0us/step


Loading a PyTorch model in TensorFlow, requires both PyTorch and TensorFlow to be installed. Please see https://pytorch.org/ and https://www.tensorflow.org/install/ for installation instructions.


ModuleNotFoundError: No module named 'torch'

In [ ]:
img_shape

(128, 128, 3)

In [ ]:
model.summary()

Model: "clip_base_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_27 (InputLayer)       [(None, 128)]                0         []                            
                                                                                                  
 input_28 (InputLayer)       [(None, 128)]                0         []                            
                                                                                                  
 input_26 (InputLayer)       [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 tf_distil_bert_model_7 (TF  TFBaseModelOutput(last_hid   1347340   ['input_27[0][0]',            
 DistilBertModel)            den_state=(None, 128, 768)   80         'input_28[0][0]']  

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20,restore_best_weights=True)

In [ ]:
from wandb.keras import WandbCallback
wandb_callback = WandbCallback(monitor='loss',
                               log_batch_frequency=10,
                               save_model = False)

In [ ]:
history = model.fit(x = train_gen, validation_data = val_gen, epochs = 100, 
                    callbacks = [early_stopping,wandb_callback],
                   workers=4)

Epoch 1/100
565/565 [==============================] - 519s 847ms/step - loss: 3.0631 - val_loss: 3.0441
Epoch 2/100
565/565 [==============================] - 450s 795ms/step - loss: 2.8296 - val_loss: 2.9169
Epoch 3/100
565/565 [==============================] - 447s 791ms/step - loss: 2.7393 - val_loss: 2.8723
Epoch 4/100
565/565 [==============================] - 443s 784ms/step - loss: 2.6881 - val_loss: 2.8591
Epoch 5/100
565/565 [==============================] - 446s 789ms/step - loss: 2.6156 - val_loss: 2.9510
Epoch 6/100
565/565 [==============================] - 443s 783ms/step - loss: 2.5907 - val_loss: 2.7975
Epoch 7/100
565/565 [==============================] - 443s 785ms/step - loss: 2.4999 - val_loss: 2.8639
Epoch 8/100
565/565 [==============================] - 442s 783ms/step - loss: 2.6099 - val_loss: 2.8272
Epoch 9/100
565/565 [==============================] - 442s 782ms/step - loss: 2.5397 - val_loss: 2.8864
Epoch 10/100
565/565 [==============================] -

In [ ]:
!mkdir -p {project_location}/weights
model.save(f"{project_location}/weights/{model_name}.h5")

In [ ]:
artifact = wandb.Artifact(model_name, type='model')
artifact.add_file(f"{project_location}/weights/{model_name}.h5")
run.log_artifact(artifact)
run.finish()